In [14]:
import tkinter as tk
import tkinter.ttk as ttk
import sells_report_auto_gen
import pandas as pd
import datetime as dt
import openpyxl

In [25]:
date = dt.datetime.strptime('2021/08/11', '%Y/%m/%d')
data = pd.read_excel("../output/sales_management.xlsx", header=0)
colname_list = ["日付", "弁当名等", "搬入個数"]  # 結果に表示させる列名
width_list = [100, 200, 50]
search_col = "日付"  # 検索キーワードの入力されている列名

workbook = openpyxl.load_workbook("../output/sales_management.xlsx")
#ワークシート指定
sheet = workbook['販売個数']

data[data['日付'] == date]
sheet.cell(row=380, column=6).value

,Unnamed: 0,日付,曜日,支店名,記録種,弁当名等,搬入個数,販売価格,650円,550円,...,350円,300円,150円,50円,販売個数,割引等単価,クーポン利用,備考,駒込で販売,売上
0,NaN,2021-06-07,NaN,御徒町,NaN,カラアゲ弁当,10.0,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2021-06-07,NaN,御徒町,NaN,サバの塩焼き弁当,6.0,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2021-06-07,NaN,御徒町,NaN,鶏のショウガ焼き弁当,5.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2021-06-07,NaN,御徒町,NaN,ハンバーグ弁当,7.0,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2021-06-07,NaN,御徒町,NaN,大盛りご飯,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,2021-06-08,NaN,御徒町,NaN,ガパオライス,5.0,NaN,NaN,3.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
